In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import cas np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from IPython.display import clear_output
!pip install imutils
clear_output()

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from keras import backend as K
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd

In [ ]:
os.listdir('../input/brain-tumor-segmented-dataset/SEG')

In [ ]:
enc = OneHotEncoder()
enc.fit([[0], [1]]) 
def names(number):
    if(number == 0):
        return 'Tumor'
    else:
        return 'Normal'

In [ ]:
data = []
paths = []
ans = []
for r, d, f in os.walk(r'../input/brain-tumor-segmented-dataset/SEG/LGG'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    x = img.resize((128,128))
    x = np.array(x)
    if(x.shape == (128,128,3)):
        data.append(np.array(x))
        ans.append(enc.transform([[0]]).toarray())

In [ ]:
paths = []
for r, d, f in os.walk(r"../input/brain-tumor-segmented-dataset/SEG/HGG"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    x = img.resize((128,128))
    x = np.array(x)
    if(x.shape == (128,128,3)):
        data.append(np.array(x))
        ans.append(enc.transform([[1]]).toarray())

In [ ]:
data = np.array(data)
data.shape

In [ ]:
ans = np.array(ans)
ans = ans.reshape(285,2)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding = 'Same'))
model.add(Conv2D(32, kernel_size=(2, 2),  activation ='selu', padding = 'Same'))


model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (2,2), activation ='selu', padding = 'Same'))
model.add(Conv2D(64, kernel_size = (2,2), activation ='selu', padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer='Adamax', metrics = ['accuracy'])
print(model.summary())

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data, ans, test_size=0.2, shuffle=True, random_state=69)

In [ ]:
history = model.fit(x_train, y_train, epochs=500, batch_size=40, verbose=1,validation_data=(x_test, y_test))

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.resnet50 import ResNet50
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

In [ ]:
IMG_SIZE = (128,128)

In [ ]:

base_model = VGG16(
    weights=None,
    include_top=False, 
    input_shape=IMG_SIZE + (3,)
)

In [ ]:
base_model = VGG16(
    weights='imagenet',
    include_top=False, 
    input_shape=IMG_SIZE + (3,), 
    #input_shape=(128,128,3)
)


In [ ]:


NUM_CLASSES = 2

model_vgg = Sequential()
model_vgg.add(base_model)
model_vgg.add(layers.Flatten())
model_vgg.add(layers.Dropout(0.5))
model_vgg.add(layers.Dense(512, activation='relu'))
model_vgg.add(layers.Dropout(0.5))
model_vgg.add(layers.Dense(2, activation='sigmoid'))

model_vgg.layers[0].trainable = False

model_vgg.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy']
)

model_vgg.summary()



In [ ]:
x_train = x_train.reshape((-1, 128, 128, 3))

In [ ]:
history = model_vgg.fit(x_train, y_train, epochs=500, batch_size=40, verbose=1,validation_data=(x_test, y_test))

In [ ]:
# history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

In [ ]:
type(x_train[0])

In [ ]:

from keras.applications import  resnet

#resnet50_weight_path = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet101 =resnet.ResNet101(include_top=False, weights='imagenet', input_shape=IMG_SIZE + (3,))


model = Sequential()
model.add(resnet101)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

model.layers[0].trainable = False
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#from keras.callbacks import ModelCheckpoint
#filepath = "saved-model-Resnet101{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

model.summary()
# resnet101 = model.fit(
#     train_generator,steps_per_epoch=50,
#     epochs=25,validation_data=validation_generator,validation_steps=25,callbacks=callbacks_list)

history = model.fit(x_train, y_train, epochs=500, batch_size=40, verbose=1,validation_data=(x_test, y_test))

In [ ]:
# history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

In [ ]:

from keras.applications import  resnet

#resnet50_weight_path = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet101 =resnet.ResNet101(include_top=False, weights=None, input_shape=IMG_SIZE + (3,))


model = Sequential()
model.add(resnet101)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

model.layers[0].trainable = False
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#from keras.callbacks import ModelCheckpoint
#filepath = "saved-model-Resnet101{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

model.summary()
# resnet101 = model.fit(
#     train_generator,steps_per_epoch=50,
#     epochs=25,validation_data=validation_generator,validation_steps=25,callbacks=callbacks_list)

history = model.fit(x_train, y_train, epochs=500, batch_size=40, verbose=1,validation_data=(x_test, y_test))

In [ ]:
# history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

In [ ]:
#after 100 epochs the accuracy was 83% on resnet 101 without using imagenet weights

In [ ]:

from keras.applications import  resnet

#resnet50_weight_path = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet101 =resnet.ResNet101(include_top=False, weights=None, input_shape=IMG_SIZE + (3,))


model = Sequential()
model.add(resnet101)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

model.layers[0].trainable = False
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#from keras.callbacks import ModelCheckpoint
#filepath = "saved-model-Resnet101{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

model.summary()
# resnet101 = model.fit(
#     train_generator,steps_per_epoch=50,
#     epochs=25,validation_data=validation_generator,validation_steps=25,callbacks=callbacks_list)

history = model.fit(x_train, y_train, epochs=500, batch_size=40, verbose=1,validation_data=(x_test, y_test))

In [ ]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
tensorboard --logdir=YOUR_LOG_DIR --host=127.0.0.1